In [1]:
from pyscenic.rnkdb import FeatherRankingDatabase, InvertedRankingDatabase
from pyscenic.genesig import GeneSignature
from typing import Type, Tuple
import os
import numpy as np
import pandas as pd
from feather.api import write_dataframe, FeatherReader
from tqdm import tqdm

In [2]:
DB_FOLDER = "/Users/bramvandesande/Projects/lcb/databases"

In [3]:
db = FeatherRankingDatabase(fname=os.path.join(DB_FOLDER, "hg19-regions-220330-9species.extracted.feather"),
                            name="regions", nomenclature="HGNC")

In [4]:
len(db.genes)

220329

In [5]:
#InvertedRankingDatabase.invert(db, os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.feather"))

In [6]:
!sort -R {DB_FOLDER}/hg19-regions-220330-9species.inverted.identifiers.txt | head -10

chr15-reg16731
chr9-reg58065
chr19-reg35363
chr1-reg105148
chr10-reg50480
chr9-reg65818
chr11-reg542
chr20-reg36490
chr7-reg91280
chr16-reg47163
sort: Broken pipe


In [7]:
from random import shuffle
with open(os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.identifiers.txt") , 'r') as f:
    ids = list(map(lambda s: s.strip(), f))
    shuffle(ids)
    gs = GeneSignature("test_regions", "regionIDs", ids[:2500])

In [8]:
inv_db = InvertedRankingDatabase(os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.feather"), "test", "regionIDs")

In [9]:
len(inv_db.genes)

220330

In [10]:
inv_db.df.head()

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
features,,,,,,,,,,,,,,,,,,,,,
elemento-AAAATGGCG,216747,146792,127978,146367,1799,192260,84271,36783,43385,45361,...,198785,81035,114643,54462,11823,14044,53253,173619,80486,95376
elemento-AAATCAAT,25546,207300,188600,128200,183402,58980,90776,63226,99184,65424,...,55706,194185,41845,144364,39523,74940,5901,192688,124017,88340
elemento-AAATGCAAA,93673,10046,29016,25672,188867,43400,130170,209852,44976,174826,...,93845,16801,199035,40073,328,42467,69899,167043,17119,138352
elemento-AAATTGCA,174175,88226,116446,31922,20921,116295,25672,216763,218405,21902,...,201940,218110,168422,219315,98532,104616,19057,210794,163261,61331
elemento-AACAGCTG,118437,116094,116772,41283,145391,206996,63244,74784,164603,31768,...,61705,135121,33140,66519,142502,135491,2230,214319,155278,115349


In [11]:
inv_db.load(gs)

,chr1-reg100393,chr1-reg100953,chr1-reg10156,chr1-reg10160,chr1-reg10224,chr1-reg102574,chr1-reg102818,chr1-reg102819,chr1-reg103151,chr1-reg103428,...,chrX-reg6785,chrX-reg7116,chrX-reg7529,chrX-reg7755,chrX-reg7958,chrX-reg9067,chrX-reg9358,chrX-reg9571,chrY-reg259,chrY-reg455
elemento-AAAATGGCG,15955,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,3638,4294967295,4294967295,...,13117,4294967295,20127,25928,4956,4294967295,4294967295,20411,25350,15622
elemento-AAATCAAT,3075,23417,4294967295,4294967295,4294967295,33635,4294967295,25372,4294967295,4294967295,...,4294967295,4294967295,30101,4294967295,235,4294967295,49267,42966,4294967295,34610
elemento-AAATGCAAA,3386,28653,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,1447,28826,4294967295,45381,330,13063,4294967295,30921,4294967295,4294967295
elemento-AAATTGCA,35427,29283,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,12482,4294967295,4294967295,42903,1036,27478,4294967295,25126,4294967295,4294967295
elemento-AACAGCTG,23342,10478,4294967295,4294967295,41232,4294967295,4294967295,4294967295,4294967295,4294967295,...,20140,4294967295,6842,14489,18224,4294967295,4294967295,29651,4294967295,4294967295
elemento-AACATGTG,4294967295,34859,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,23023,4294967295,...,10162,4294967295,4294967295,31377,4294967295,4294967295,4294967295,29981,4294967295,41495
elemento-AACCGGTT,3647,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,47501,46835,4294967295,14291,5903,4294967295,16459,4294967295,35888,26820
elemento-AACGTGA,29505,25753,4294967295,4294967295,4294967295,4294967295,4294967295,28480,4294967295,4294967295,...,4294967295,4294967295,12215,4294967295,43391,14137,18243,4294967295,35112,4294967295
elemento-AACGTGC,22001,14622,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,9915,...,4294967295,4294967295,25025,31394,30457,47340,4294967295,4294967295,42801,4294967295
elemento-AAGAAGGC,7510,34858,4294967295,30835,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,13981,23250,4294967295,34087,4294967295,4294967295,4294967295,4294967295
